This notebook is currently in progress

# GEE Aster Preprocessing

## Step 1: Create a google earth engine account

Follow this [link](https://code.earthengine.google.com/register) to create a GEE account

## Step 2: Installation


In [1]:
%pip install git+https://github.com/Mining-for-the-Future/ASTER_preprocessing.git@function_disaggregation

  Cloning https://github.com/Mining-for-the-Future/ASTER_preprocessing.git (to revision function_disaggregation) to c:\users\eweav\appdata\local\temp\pip-req-build-wego8_am
  Resolved https://github.com/Mining-for-the-Future/ASTER_preprocessing.git to commit 99d7b7c28603957fa35eb13965582aa431a63f53
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/Mining-for-the-Future/ASTER_preprocessing.git 'C:\Users\eweav\AppData\Local\Temp\pip-req-build-wego8_am'
  Running command git checkout -b function_disaggregation --track origin/function_disaggregation
  branch 'function_disaggregation' set up to track 'origin/function_disaggregation'.
  Switched to a new branch 'function_disaggregation'


## Step 3: Importing Libraries

In [ ]:
%pip install geemap

importing these modules should trigger the google earth engine authentication process.

Note: in VS Code, the input field for the google earth engine token is at the top of the window, not underneath the cell.

In [2]:
from ASTER_preprocessing.preprocessing import aster_collection_preprocessing, aster_image_preprocessing
from ASTER_preprocessing.vis_params import vis_params_image


Successfully saved authorization token.


In [3]:
import geemap

## Step 4: Preprocess ASTER image collection using a drawn polygon

In [4]:
# Create an interactive map
Map = geemap.Map()
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [5]:
# Get the last drawn feature
geometry = Map.user_roi

In [6]:
collection = aster_collection_preprocessing(geometry)

In [8]:
from ASTER_preprocessing.preprocessing import ee_i

In [24]:
raw_collection = ee_i.ImageCollection("ASTER/AST_L1T_003").filterBounds(geometry).filter(ee_i.Filter.lte('CLOUDCOVER', 25))
raw_collection.size()

In [28]:
map2 = geemap.Map()
map2.centerObject(geometry)
map2.add_time_slider(raw_collection, vis_params = {'bands': ['B04', 'B3N', 'B02'], 'min': [0, 0, 0], 'max': [255, 255, 255]})

No such comm: 15278e4b09cc4531b9c38bdb5dbe86ac
No such comm: fa97db77d789494c8f481eeff086011a


In [30]:
map2.addLayer(geometry)
map2

Map(bottom=190105.0, center=[44.411515756271726, -93.1019897463557], controls=(WidgetControl(options=['positio…

In [31]:
raw_coll_list = raw_collection.toList(count = raw_collection.size())

In [32]:
raw_coll_list

In [33]:
if ee_i.List([]).length().eq(0):
    print("Python recognized an empty ee.List")
else:
    print("Nope, couldn't recognize empty list")

Python recognized an empty ee.List


In [7]:
collection.size().getInfo()

38

In [9]:
from ASTER_preprocessing.ee_utm_projection import get_utm_proj_from_coords

In [10]:
def get_geom_area(geom, proj):
   return geom.area(maxError = 1, proj = proj)

def get_pixel_area(image, geom, proj):
   return ee_i.Number(image.pixelArea().reduceRegion(ee_i.Reducer.sum(), geom, crs = proj.crs(), scale = proj.nominalScale(), bestEffort = True).get('area'))

In [14]:
def set_geom_coverage_property(image, geom, geom_area, proj):
   return image.set({'geom_coverage': get_pixel_area(image, geom, projection).divide(geom_area)})

In [12]:
projection = get_utm_proj_from_coords(geometry.centroid(maxError = 1).coordinates().getInfo())

In [17]:
image_list = collection.toList(count = collection.size())

In [18]:
image_list.get(0).getInfo()

EEException: Collection.toList: Error in map(ID=20070904040544):
Image.constant: Parameter 'value' is required.

In [22]:
map_id = "20070904040544"
weird_image = ee_i.Image(map_id)

map = geemap.Map()
map.addLayer(weird_image)
map.centerObject(weird_image)
map

EEException: Image.load: Image asset '20070904040544' not found (does not exist or caller does not have access).

In [ ]:
set_geom_coverage_property(collection.first(), geometry, geom_area = get_geom_area(geometry, proj = projection), proj = projection).getInfo()

In [41]:
expected_area = geometry.area(maxError = 1, proj = get_utm_proj_from_coords(geometry.centroid(maxError = 1).coordinates().getInfo())).getInfo()

In [32]:
30**2

900

In [39]:
proj = get_utm_proj_from_coords(geometry.centroid(maxError = 1).coordinates().getInfo())

In [50]:
actual_area = collection.first().pixelArea().reduceRegion(reducer = ee_i.Reducer.sum(), geometry = geometry, crs = proj.crs(), scale = 30, bestEffort = True).get('area').getInfo()

In [51]:
actual_area

70930251.50048856

In [52]:
actual_area / expected_area

1.000780632555122

In [35]:
collection.first().reduceRegion(ee_i.Reducer.count(), geometry, scale = 30, bestEffort = True).getInfo()

In [120]:
geom = geometry
projection = get_utm_proj_from_coords(geom.centroid(maxError = 1).coordinates().getInfo())
area = get_geom_area(geom, projection)

In [122]:
px_area = get_pixel_area(collection.first(), geom, projection)

In [123]:
image = collection.first()

In [124]:
image

In [ ]:
image.set({'geom_coverage': get_pixel_area(image, geom, projection).divide(area)})

In [133]:
collection1 = collection.map(lambda x: set_geom_coverage_property(image = x, geom = geometry, geom_area = area, projection = projection))

In [134]:
collection1.getInfo()

EEException: Error in map(ID=20070904040544):
Image.constant: Parameter 'value' is required.

In [109]:
px_area.divide(area)

In [58]:
geom_area = get_geom_area(geometry)

In [57]:
px_areas = get_pixel_area(collection.first(), geometry)

In [60]:
px_areas.map(lambda key, value: ee_i.Number(value).divide(ee_i.Number(geom_area)))

In [61]:
map = geemap.Map()
map.addLayer(collection.first())
map.centerObject(collection.first())
map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [54]:
get_geom_area(geometry)

70874924.22729492

## Step 4A: preprocess aster image from drawn point

In [ ]:
Map = geemap.Map()
Map

In [ ]:
geometry = Map.user_roi

### Examine the dates of images intersecting this point to select one

In [ ]:
from ASTER_preprocessing.preprocessing import ee_i

In [ ]:
from datetime import datetime

In [ ]:
collection = ee_i.ImageCollection('ASTER/AST_L1T_003').filterBounds(geometry)

In [ ]:
image_list = collection.toList(collection.size())

In [ ]:
dates = []
for i in range(image_list.length().getInfo()):
    image = ee_i.Image(image_list.get(i))
    date = datetime.utcfromtimestamp(image.get('system:time_start').getInfo()/1000).strftime('%Y-%m-%d')
    dates.append(date)

In [ ]:
dates

In [ ]:
collection.filterDate('2005-03-26', '2005-03-27').size().getInfo()

In [ ]:
image = collection.filterDate('2005-03-26', '2005-03-27').first()

In [ ]:
def draw_map(img):
    map = geemap.Map()
    map.centerObject(img)
    map.addLayer(img)
    return map

In [ ]:
draw_map(image)

In [ ]:
image.bandNames().getInfo()

In [ ]:
image.get('CLOUDCOVER').getInfo()

In [ ]:
from ASTER_preprocessing.data_conversion import aster_dn2toa

In [ ]:
image_toa = aster_dn2toa(image, ['B3N', 'B04', 'B05'])
map = geemap.Map()
map.center_object(image_toa)
map.addLayer(image_toa, vis_params = {'bands': ['B3N', 'B04', 'B05'], 'min': [0, 0, 0], 'max': [1, 1, 1]})
map

In [ ]:
image_proc = aster_image_preprocessing(image, ['B3N', 'B02', 'B01'], masks = ['cloud'])
vis_params = vis_params_image(image_proc, ['B3N', 'B02', 'B01'])

map = geemap.Map()
map.center_object(image_proc)
map.addLayer(image_proc, vis_params = vis_params, name = 'processed_image')
map

In [ ]:
image = aster_image_preprocessing(image, bands = ['B3N', 'B04', 'B05'], masks = [])

In [ ]:
image.bandTypes().getInfo()

In [ ]:
vis_params_image(image, ['B3N', 'B04', 'B05'])

In [ ]:
def vis_params_image(image, bands):
    geometry = image.geometry()
    projection = image.select(bands[0]).projection().getInfo()
    
    percentiles = image.select(bands).reduceRegion(
        reducer = ee_i.Reducer.percentile([5, 95]),
        geometry = geometry,
        crs = projection['crs'],
        crsTransform = projection['transform'],
        bestEffort = True).getInfo()

    b_p5 = [band + '_p5' for band in bands]
    b_p95 = [band + '_p95' for band in bands]
    mins = [percentiles[min] for min in b_p5]
    maxs = [percentiles[max] for max in b_p95]
    vis_params = {'min': mins, 'max': maxs, 'bands': bands}
    return vis_params

In [ ]:
image.geometry().getInfo()

In [ ]:
map = geemap.Map()
map.centerObject(image)
map.addLayer(image)
map

In [ ]:
image.select('B01').projection().getInfo()

## Step 5: Visualize the image collection as a time series

In [20]:
first_image = collection.first()
vis_params = vis_params_image(first_image, bands = ['B04', 'B3N', 'B02'])

map2 = geemap.Map()
map2.centerObject(first_image)
# map2.addLayer(first_image, vis_params)
map2.add_time_slider(collection, vis_params = vis_params, time_interval = 1)
map2

EEException: ImageCollection.toBands: Error in map(ID=20070904040544):
Image.constant: Parameter 'value' is required.